In [1]:
import pandas as pd

from tqdm.notebook import tqdm

# Import SparkConf class into program
from pyspark import SparkConf

# local[*]: run Spark in local mode with as many working processors as logical cores on your machine
# If we want Spark to run locally with 'k' worker threads, we can specify as "local[k]".
master = "local[*]"
# The `appName` field is a name to be shown on the Spark cluster UI page
app_name = "Parallel Join"
# Setup configuration parameters for Spark
spark_conf = SparkConf().setMaster(master).setAppName(app_name)

# Import SparkSession classes 
from pyspark.sql import SparkSession # Spark SQL

spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()

22/12/15 23:46:54 WARN Utils: Your hostname, crarojasca-Blade-14-RZ09-0370 resolves to a loopback address: 127.0.1.1; using 192.168.1.10 instead (on interface wlp2s0)
22/12/15 23:46:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/15 23:46:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
file = "datasets/omm_export_tweets_01-06-2022.csv"
data = spark.read.options(delimiter="\t").csv(file, header=True)

In [3]:
data.printSchema()

root
 |-- id: string (nullable = true)
 |-- username: string (nullable = true)
 |-- fulltext: string (nullable = true)
 |-- 140_char_text: string (nullable = true)
 |-- hashtags: string (nullable = true)
 |-- url: string (nullable = true)
 |-- domain: string (nullable = true)
 |-- retweet_count: string (nullable = true)
 |-- favorite_count: string (nullable = true)
 |-- language: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lon: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_location: string (nullable = true)
 |-- user_time_zone: string (nullable = true)
 |-- user_follower_count: string (nullable = true)
 |-- user_favorite_count: string (nullable = true)
 |-- user_tweet_count: string (nullable = true)
 |-- user_description: string (nullable = true)
 |-- media: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: 

In [4]:
sample = data.sample(0.05)

In [5]:
sample.toPandas().to_csv("datasets/omm_export_tweets_sampled.csv")

In [14]:
import torch
from torch.utils.data import Dataset, DataLoader

import pickle
from scipy.special import softmax
from utils import roberta_model, denoise_text

import linecache

class HumDataset(Dataset):
    def __init__(self, file):
        self.file = file
        self.data = spark.read.options(delimiter="\t").csv(file, header=True)
        self.length = self.data.count() - 2
  
    def __len__(self):
        return self.length
 
    def __getitem__(self, idx):
        line = linecache.getline("datasets/omm_export_tweets_01-06-2022.csv", idx+2)
        text = denoise_text(line.split("\t")[3])
        return text 

In [ ]:
dataset = HumDataset("datasets/omm_export_tweets_01-06-2022.csv")
dataloader = DataLoader(dataset, batch_size=50, shuffle=False)


with open('cards/models/label_encoder.pkl', 'rb') as f:
    le = pickle.load(f)

for batch in tqdm(dataloader):
    print(batch)
    predictions, raw_outputs = roberta_model.predict(batch)
    print(predictionspredictions)
    break

  0%|          | 0/104734 [00:00<?, ?it/s]

['Melinda is switching from @HESTASuper to another superannuation fund after HESTA didnt care when she voiced concer... https://t.co/jf12KZBRwB', 'How does the #UkraineCrisis, the #pandemic, and #ClimateChange affect the #GlobalSouths demand for #energy and its... https://t.co/dq3xVS8ekJ', 'Climate Change: Tech Billionaire Wins Battle Against Australias Biggest Polluter https://t.co/usgs5KUgcT', 'The newest drinking public#millennials and #GenZrate climate change and the environment as their top concern, and... https://t.co/Utw39xrKcQ', 'With two-thirds of the impact of aviation on climate change believed to come from aircraft non-carbon-dioxide emiss... https://t.co/b4tWu6WCoI', 'Farmers battling climate change welcome Labor, Greens stronger policies https://t.co/1apCS7ojj5 https://t.co/rzsv9dv0qt', 'Taking Root is #hiring! We have 4 openings in our Product and Engineering department, so if you want to make a m... https://t.co/gS3CFi4e3c', 'And watch Bolt, pleading, begging with Dutto

In [18]:







data['roberta_pred'] = le.inverse_transform(predictions)
data['roberta_proba'] = [max(softmax(element[0])) for element in raw_outputs]

data.to_csv("datasets/omm_export_tweets_sampled_scored.csv", index=False)

KeyboardInterrupt: 